In [ ]:
from glob import glob
import importlib
from sklearn.model_selection import ParameterGrid
from symbolic_algs import symbolic_algs

sr_frames = []
ml_frames = []

for m in [n.split('/')[-1][:-3] for n in glob('methods/*.py')]:
    if m.startswith('_'): continue
    if 'EHC' in m: continue
    if 'FFX' in m: continue
    
    m_short = m.replace('Regressor','')
    
    alg = importlib.__import__('methods.'+m,
                               globals(),locals(),                                  
                               ['*']
                              )  
    if m_short == 'DSR':
        hp = {'batch_size':alg.batch_sizes}
    else:
        hp = alg.hyper_params
    if isinstance(hp, list):
        tuneables = hp[0].keys()
    else:
        tuneables = hp.keys()
    
#     print(m)
#     print(hp)
    
    # get tuned for symbolic_algs
    if m not in symbolic_algs:
        if isinstance(hp, list):
            for p in hp:
                ml_frames.append([m_short, p])
        else:
            ml_frames.append([m_short, hp]) # hp_tuned])
        continue
    else:
        alg_tuned = importlib.__import__('methods.tuned.params._'+m.lower(),
                                   globals(),locals(),                                  
                                   ['params']
                                  )  
        hp_tuned = {k:v for k,v in alg_tuned.params.items() if k in tuneables}
        if isinstance(hp, list):
            for p in hp:
                sr_frames.append([m_short, p])
        else:
            sr_frames.append([m_short, hp]) # hp_tuned])
        
#     hp_pg = ParameterGrid(hp)
#         print('tuned:',hp_tuned)
#     print(list(hp_pg))
print('ML')
print(ml_frames)

In [ ]:
import pdb
import json
from utils import jsonify
def hp_format(p):
#     pdb.set_trace()
    if isinstance(p, dict):
        for k,v in p.items():
            if isinstance(v, list) and len(v)==1:
                p[k] = v[0]
            elif isinstance(v, float):
                p[k] = round(v,2)
    elif isinstance(p,list):
        return [hp_format(i) for i in p]
            
    return p
#     args = dict(sort_keys=True, indent=4)
# #     if isinstance(p, list):
# #         return '\\\\'.join([json.dumps(jsonify(i), **args)
# #                           for i in p])
# #     else:
#     return json.dumps(jsonify(p), **args)

def add_midrules(ltx):
    ltx2 = ltx.splitlines()
    size = len(ltx2) - 1
    for i,line in enumerate(ltx.splitlines()[::-1]):
        method = line.split('&')[0].strip().replace('\\','')
        if method in methods: 
#             pdb.set_trace()
            ltx2.insert(size-i,'\midrule')
    
    return '\n'.join(ltx2)

In [ ]:
methods

In [ ]:
import pandas as pd
from math import floor, ceil
column_format = 'l p{37em}'
pd.set_option('display.max_colwidth', 1000)
sr_df = pd.DataFrame(sr_frames, columns = ['Method','Hyperparameters']).sort_values(by='Method')
methods = sr_df.Method.unique()
for i in [1,2]:
    print_methods = methods[:ceil(len(methods)/2)] if i == 1 else methods[ceil(len(methods)/2):]
    df = sr_df.loc[sr_df.Method.isin(print_methods)]
    df.loc[df['Method'] == df['Method'].shift(1), 'Method'] = ''
    sr_ltx = df.to_latex(
                index=False,
                column_format=column_format,
                float_format='{:0.1f}',
                formatters={
                    'Hyperparameters': hp_format
                       }
               )
    # insert midrule between methods
    sr_ltx = add_midrules(sr_ltx)
#     for s in sr_ltx.splitlines(): print(s[:80])
    with open('../../paper/tables/sr_methods_hp_table'+str(i)+'.tex','w') as f:
        f.write(sr_ltx)
        
##########
# ML table
##########
ml_df = pd.DataFrame(ml_frames, columns = ['Method','Hyperparameters'])
ml_df.loc[ml_df['Method'] == ml_df['Method'].shift(1), 'Method'] = ''
ml_ltx = ml_df.to_latex(
                        index=False,
                        column_format=column_format,
                        float_format='{:0.1f}',
                        formatters={
                            'Hyperparameters': hp_format
                       }
           )
# insert midrule between methods
ml_ltx2 = ml_ltx.splitlines()
for i in range(len(ml_ltx),0):
    ml_ltx2.insert(i,'\midrule') 
ml_ltx = '\n'.join(ml_ltx2)
    #add_midrules(sr_ltx)
with open('../../paper/tables/ml_methods_hp_table.tex','w') as f:
    f.write(ml_ltx)

In [ ]:
for s in ml_ltx.splitlines(): print(s[:80])

In [ ]:

add_midrules(sr_ltx)


In [ ]:
ml_df